In [ ]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run UCBM_setup.ipynb

from collections import defaultdict

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

import math
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))

UCTBmodel = UCBM_model()

In [ ]:
del.T,  vtype=GRB.CONTINUOUS, name='w_curtail')

# line power rate# create one model instance, with a name
UCTBM_gurobi = gp.Model("UC_TwoBinary_model")

# continuous decision variables
# total cost that includes the first-stage UC cost of the regular units
phi = UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.CONTINUOUS, name='phi')
#phi_su = UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.CONTINUOUS, name='phi_su')

# power output from unit i in time period t.
p = UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.CONTINUOUS, name='p')

# curtailment from renewable energy
#w_curtail = UCTBM_gurobi.addVars(UCTBmodel.Wb, UCTBmo
#f = UCTBM_gurobi.addVars(len(UCTBmodel.Tx_list), UCTBmodel.T, lb=-float('inf'),  vtype=GRB.CONTINUOUS, name='f')
# line power rate adjust to positive
ff = UCTBM_gurobi.addVars(len(UCTBmodel.Tx_list), UCTBmodel.T, vtype=GRB.CONTINUOUS, name='ff')
bf = UCTBM_gurobi.addVars(len(UCTBmodel.Tx_list), UCTBmodel.T, vtype=GRB.CONTINUOUS, name='bf')

# phase of each line
#theta = UCTBM_gurobi.addVars(UCTBmodel.Bus, UCTBmodel.T, lb=-float('inf'),  vtype=GRB.CONTINUOUS, name='theta')
# phase of each line adjust to positive
theta = UCTBM_gurobi.addVars(UCTBmodel.Bus, UCTBmodel.T, vtype=GRB.CONTINUOUS, name='theta')



# continuous state variables
# The expected second-stage dispatch cost under the worst-case probability distribution
#phi_su =UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.CONTINUOUS, name='phi_su')


# binary continuous variables
# charging and discharging of battery state

u = UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.BINARY, name='u')

v = UCTBM_gurobi.addVars(UCTBmodel.G_I, UCTBmodel.T, vtype=GRB.BINARY, name='v')

# objective function
# phi = quicksum(UCTBmodel.SU[i] * (u[i,t] - u[i,t - 1] )

#phi_su[i,t]
UCTBM_gurobi.setObjective(quicksum( 
                                quicksum(UCTBmodel.G_ST_cost[i] * v[i,t] +  phi[i,t]  
                                        for t in range(1, UCTBmodel.T)) 
                                for i in range(UCTBmodel.G_I)), GRB.MINIMIZE)


    
# constarint 1
for i in range(UCTBmodel.G_I):
    
    #Initial the Generators' status
    UCTBM_gurobi.addConstr(u[i, 0] == UCTBmodel.u_0[i]*0,  name=f'UCTBmodel_Init_u_GI_{i}')

    # constarint 1
    for t in np.arange(1, UCTBmodel.T):
        
        UCTBM_gurobi.addConstr(u[i,t] - u[i,t - 1] <=  v[i, t], name=f'UCTBmodel_cons_0_GI_{i}_T_{t}')
    
    # constarint 2a
    for t in np.arange(UCTBmodel.UT[i], UCTBmodel.T):
        UCTBM_gurobi.addConstr(quicksum( v[i,k] for k in np.arange(t - UCTBmodel.UT[i] + 1 , t + 1))  <= u[i,t]
                               , name=f'UCTBmodel_cons_1_GI_{i}_T_{t}')
        
    # constarint 2b
    for t in np.arange(UCTBmodel.DT[i], UCTBmodel.T):
        UCTBM_gurobi.addConstr(
            quicksum( v[i,k] for k in np.arange(t - UCTBmodel.DT[i] + 1, t + 1))  <= 1 - u[i,t - UCTBmodel.DT[i]]
                               , name=f'UCTBmodel_cons_2_GI_{i}_T_{t}')

for i in range(UCTBmodel.G_I):
    for t in np.arange(1, UCTBmodel.T):
        
        # Constraint 3
        
        ## cost_fix : start up cost coefficient, fix cost for u $Gamma$
        ## costOf : start up cost coefficient, fix cost for p $Beta$
        
        # General model Beta and gamma needs J intervals of Piesce-wise linear model
        #for j in np.range(UCTBmodel.J):
        #    UCTBM_gurobi.addConstr(UCTBmodel.costOf[i,j] * p[i,t] + UCTBmodel.cost_fix[i,j] * u[i,t] - phi[i,t] <= 0 
        #                           , name=f'UCTBmodel_cons_3_J_{j}_T_{t}')
        
        #However, our set up the the cost Function is linear, which means the coefficient for the quadratic term is 0
        
        #UCTBM_gurobi.addConstr(phi_su[i,t] - UCTBmodel.G_ST_cost[i] * v[i,t] >= 0 
        #                       , name=f'UCTBmodel_cons_add_GI_{i}_T_{t}')
        
        UCTBM_gurobi.addConstr(- UCTBmodel.costOf[i] * p[i,t] - UCTBmodel.cost_fix[i] * u[i,t] + phi[i,t] >= 0 
                               , name=f'UCTBmodel_cons_3_GI_{i}_T_{t}')
        
           
        # Constraint 4
        UCTBM_gurobi.addConstr(UCTBmodel.P_UB[i] * u[i,t] - p[i,t] >= 0  , name=f'UCTBmodel_cons_4_GI_{i}_T_{t}_LB')
        UCTBM_gurobi.addConstr(p[i,t] - UCTBmodel.P_LB[i] * u[i,t] >= 0 , name=f'UCTBmodel_cons_4_GI_{i}_T_{t}_UB')
        
        # Constraint 5
        
        UCTBM_gurobi.addConstr(UCTBmodel.RU_O_UB[i] * u[i,t - 1] + UCTBmodel.RU_CS_UB[i] * (1 - u[i,t - 1]) - p[i,t] + p[i,t - 1] >= 0,
                                   name=f'UCTBmodel_cons_5_GI_{i}_T_{t}')
            
        # Constraint 6
        UCTBM_gurobi.addConstr(UCTBmodel.RD_O_UB[i] * u[i,t] + UCTBmodel.RD_HS_UB[i] * (1 - u[i,t]) - p[i,t - 1] + p[i,t] >= 0 ,
                                   name=f'UCTBmodel_cons_6_GI_{i}_T_{t}')
        
        # note: Due to some concerns we decide to let w^{cur}_{i,t} stay at 0 at first. 
        # Once we find the wind patterns we will put it back to the real value


for t in np.arange(1, UCTBmodel.T):

    #for wb_ in range(UCTBmodel.Wb):
        # Constraint 7
        #UCTBM_gurobi.addConstr(w_curtail[wb_,t] >= 0, name=f'UCTBmodel_cons_7_Wb_{wb_}_T_{t}_LB')
        #UCTBM_gurobi.addConstr(w_curtail[wb_,t] <= 0 * UCTBmodel.Wb_p_list[wb_], name=f'UCTBmodel_cons_7_Wb_{wb_}_T_{t}_UB')

    
    for b_ in UCTBmodel.Bus_list:
        # Constraint 8 f: from t:toward
        f_keys = [item for item in UCTBmodel.Tx_list if item[1] == b_ ]
        f_Bus_index = [UCTBmodel.Tx_list.index(key) for key in f_keys] 
            
        t_keys = [item for item in UCTBmodel.Tx_list if item[0] == b_ ]
        t_Bus_index = [UCTBmodel.Tx_list.index(key) for key in t_keys]    
            
        #wb_index = np.where(UCTBmodel.Wb_list == b_)[0]
                
        Gen_index = np.where(UCTBmodel.G_I_list == b_)[0]
            
        #print(b_ , Gen_index,f_Bus_index,t_Bus_index)
            
        #UCTBM_gurobi.addConstr(quicksum(f[k,t] for k in f_Bus_index) - quicksum(f[k,t] for k in t_Bus_index) + 
        #                        0 * quicksum(UCTBmodel.Wb_p_list[k] - w_curtail[k,t] for k, item in enumerate(wb_index)) +
        #                        quicksum(p[item,t] for item in Gen_index)  == UCTBmodel.Bus_Pdt[int(b_) - 1, t - 1], name=f'UCTBmodel_cons_8_b_{b_}_T_{t}')
        
        UCTBM_gurobi.addConstr(quicksum(ff[k,t] - bf[k,t] for k in f_Bus_index) - quicksum(ff[k,t] - bf[k,t] for k in t_Bus_index) + 
                                quicksum(p[item,t] for item in Gen_index)  == UCTBmodel.Bus_Pdt[int(b_) - 1, t - 1], name=f'UCTBmodel_cons_8_b_{b_}_T_{t}')
        
        UCTBM_gurobi.addConstr(theta[int(b_)-1, t] <= 2*np.pi,  name=f'UCTBmodel_cons_12_B_{b_}_T_{t}A')
        #UCTBM_gurobi.addConstr(theta[int(b_)-1, t] >= 0,  name=f'UCTBmodel_cons_12_B_{b_}_T_{t}B')
        
        
    
    for l_, (m_, n_) in enumerate(UCTBmodel.Tx_list):
        
        # Constraint 9 m: from n:toward l_:line_index 
        #print(l_, m_, n_)
        UCTBM_gurobi.addConstr(UCTBmodel.Tx_impd_list[l_] * (ff[l_, t] - bf[l_, t]) == (theta[int(m_) - 1, t] - theta[int(n_) - 1, t]), name=f'UCTBmodel_cons_9_l_{l_}_T_{t}')
           
        UCTBM_gurobi.addConstr( bf[l_, t] <= UCTBmodel.Tx_cap_list[l_], name=f'UCTBmodel_cons_10_l_{l_}_T_{t}_LB')
        UCTBM_gurobi.addConstr( ff[l_, t] <= UCTBmodel.Tx_cap_list[l_], name=f'UCTBmodel_cons_10_l_{l_}_T_{t}_UB')
    

    #Slack bus 1
    UCTBM_gurobi.addConstr(theta[0, t] - np.pi == 0,  name=f'UCTBmodel_SlackBus0_11_T_{t}')


#UCTBM_gurobi.feasRelaxS(2, False,  False, True)
#vars = UCTBM_gurobi.getVars()
#ubpen = 100 * np.ones(UCTBM_gurobi.numVars)
#lbpen= ubpen
#UCTBM_gurobi.feasRelax(2, False, vars, lbpen, ubpen, None, None)

UCTBM_gurobi_sol = UCTBM_gurobi.optimize()
print(UCTBM_gurobi.getAttr(GRB.Attr.Status))
#assert UCTBM_gurobi_sol
#UCTBM_gurobi.display()



#UCTBM_gurobi.computeIIS()
UCTBM_gurobi.write("LP_check_2bin.lp")